### Obtencion de datos 
--- 
Se obtendran  los datos de la pagina del presidente (https://presidente.gob.mx/sala-de-prensa/boletines/) usando selenium

In [15]:
#se importan las librerias necesarias para el funcionamiento del programa
from selenium import webdriver
from bs4 import BeautifulSoup
import re
import time

#### Se inicia el navegador
--- 

In [39]:
# Configuración del driver de Selenium para abrir el navegador

#se pede usar en diferentes navegadores
'''
driver= webdriver.ChromiumEdge(executable_path='msedgedriver.exe')
driver = webdriver.Firefox(executable_path='/path/to/geckodriver')
'''
driver= webdriver.Chrome(executable_path='chromedriver.exe')
driver.get('https://presidente.gob.mx/sala-de-prensa/boletines/')

# Función para obtener los links de los boletines de prensa dependiendo de la página en la que se encuentre
def navegacion(num):
    if( num >= 2):
        driver.get('https://presidente.gob.mx/sala-de-prensa/boletines/page/'+ str(num) +'/')
    if (num == 1):
        driver.get('https://presidente.gob.mx/sala-de-prensa/boletines/')

    return driver

c:\Users\te524132\AppData\Local\miniconda3\envs\topic_modeling\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  


#### Se itera en los articulos de cada pagina y se guarda en un archivo_con_articulos de texto
--- 

In [38]:
def escribir_articulos(archivo,titulo,fecha,articulo=None):
    # Escribir el título y contenido del artículo en el archivo_con_articulos de texto
    archivo.write("\n{\n")
    archivo.write(f"\"Título\": \"{titulo}\",\n")
    if articulo is not None:
        archivo.write(f"\"Fecha\": \"{fecha}\",\n")
        archivo.write(f"\"Contenido\": \"{articulo}\"")
    else:
        archivo.write(f"\"Fecha\": \"{fecha}\"")
    archivo.write("\n},")

# Abrir un archivo_con_articulos de texto para escribir los datos recolectados en cada una de las paginas de los articulos
with open('juve_datos_articulos.txt', 'w', encoding='utf-8') as archivo_con_articulos:
    with open('juve_datos.txt', 'w', encoding='utf-8') as archivo_sin_articulos:
        archivo_con_articulos.write("{")
        archivo_sin_articulos.write("{")

        #este ciclo se encarga de recorrer las paginas de los articulos de los boletines de prensa
        #solo se tiene que modificar el rango pra tomar mas paginas
        for num in range(1,316):
            navegacion(num)
            print("pagina: " + driver.current_url)
            
            # Obtener el HTML de la página con BeautifulSoup
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            
            # Obtener todos los artículos de la página en la que se encuentra y su fecha
            articulos = soup.find_all('h3', {'class': 'entry-title'})
            fecha_articulo = soup.find('span', {'class': 'posted-on'}).text.strip()
            
            # Recorrer cada artículo y obtener su contenido de cada una de los articulos de los boletines de prensa
            for articulo in articulos:
                enlace = articulo.find('a')['href']
                driver.get(enlace)
                soup_articulo = BeautifulSoup(driver.page_source, 'html.parser')
                titulo_articulo = soup_articulo.find('h2', {'class': 'entry-title'}).text.strip()
                contenido_articulo = soup_articulo.find('div', {'class': 'entry-content'}).text.strip()

                # Escribir el título y contenido del artículo en el archivo_con_articulos de texto
                escribir_articulos(archivo_con_articulos,titulo_articulo,fecha_articulo,contenido_articulo)
                escribir_articulos(archivo_sin_articulos,titulo_articulo,fecha_articulo)

            print("ciclo de boletines: " + str(num))

        archivo_con_articulos.write("\n}")        
        archivo_sin_articulos.write("\n}")        

# Cerrar el driver al finalizar y cerrar el archivo_con_articulos de 
# Mientras se esten haciendo pruebas, recomiendo comentar estas dos lineas
driver.quit()
archivo_con_articulos.close()
archivo_sin_articulos.close()

pagina: https://presidente.gob.mx/sala-de-prensa/boletines/
ciclo de boletines: 1
